In [1]:
import re
import csv
import nltk
import json
import time
import pickle
import tweepy
import pandas as pd
import nltk.classify
from tweepy import OAuthHandler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from tweepy.streaming import StreamListener
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
def removeTagsAndLinks(tweet):
    cleaned_tweet=re.sub("@[A-Za-z0-9]+|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",tweet)
    return cleaned_tweet

In [3]:
def tokenise(tweet):
    cleaned_tweet=removeTagsAndLinks(tweet)
    sentences=sent_tokenize(cleaned_tweet)
    word=RegexpTokenizer(r'\w+')
    words=[word.tokenize(sentence) for sentence in sentences]
    return words

In [4]:
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1\1", s)

def processTweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet = re.sub('@[^\s]+','AT_USER',tweet)    
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    return tweet


def getStopWordList(stopWordListFileName):
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(r'stopwordsID.txt', 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords


def getFeatureVector(tweet, stopWords):
    featureVector = []  
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w) 
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*[a-zA-Z]+[a-zA-Z0-9]*$", w)
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector    

def removeStopwords(tweet):
    stop_words=set(stopwords.words('indonesian'))
    filtered_tweet=[word for dummytweet in tweet for word in  dummytweet  if word not in stop_words]
    return filtered_tweet

In [5]:
def convert(tweet):
    converted_tweet=''
    for word in tweet:
        converted_tweet+=word+" "
    return converted_tweet


In [6]:
def authorize():
    cunsomer_key = 'd3GCOcnCQVrHqYdbkA6p5B0sb'
    consumer_secret = '3XqfMNq4Aj6rnuUjjV6ZCiQdt4SPeWG6N5l5CcrXf1C540vpnV'
    access_token = '772637934390345728-ZtWEKm6pab38OFDtXqlRgjcEvmdRZ9v'
    access_token_secret = 'MHx06gmb5SC4EeXIkvJFKMwyxZD3PMDBOGoGyH6j6fC6A'
    auth = tweepy.OAuthHandler(cunsomer_key , consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

In [7]:
def analyze(query,limit=100000,language='in'):
    list_tweet = []
    list_kelas = []
    summary={'Positive':0,'Negative':0,'Neutral':0}
    api=authorize()
    sid=SentimentIntensityAnalyzer()
    
    tweets=tweepy.Cursor(api.search,q=query,lang=language).items(limit)
        
    for tweet in tweets:
        tweet_text= tweet.text
        tweet=tokenise(tweet_text)
        tweet=removeStopwords(tweet)
        tweet=convert(tweet)
        score=sid.polarity_scores(tweet)
        sentiment = ''
        if score['compound']==0:
            summary['Neutral']+=1
            sentiment = 'Neutral'
        elif score['compound']>0:
            summary['Positive']+=1
            sentiment = 'Positive'
        else:
            summary['Negative']+=1
            sentiment = 'Negative'
        list_tweet.append(tweet)
        list_kelas.append(sentiment)
    for keys in summary.keys():
        print("No of %s tweets are %d"%(keys,summary[keys]))
    df = pd.DataFrame({'tweet': list_tweet,'kelas': list_kelas})
    df.to_csv('dataset.csv', sep='\t', encoding='utf-8', index=False)

In [8]:
def main():
    query=input('Enter the key to be searched:')
    no=int(input('Enter the number of tweets to be analyzed:'))
    try:
        language=input('Enter the language:')
    except:
        language='in'
    print('====================================================')
    analyze(query,no,language)
if __name__=='__main__':
    main()

Enter the key to be searched:dpr
Enter the number of tweets to be analyzed:100
Enter the language:in
No of Positive tweets are 6
No of Negative tweets are 0
No of Neutral tweets are 94


In [9]:
def df_test():
    negative_tweets = (df_test[df_test.sentiment==0].text).tolist()
    neutral_tweets = (df_test[df_test.sentiment==2].text).tolist()
    positive_tweets = (df_test[df_test.sentiment==4].text).tolist()

In [10]:
negative_tweets=[]
for string in negative_tweets:
    words=nltk.word_tokenize(string.decode("utf-8"))
    negative_tweets.append((create_word_features(words),'positive'))
    
neutral_tweets=[]
for string in neutral_tweets:
    words=nltk.word_tokenize(string.decode("utf-8"))
    neutral_tweets.append((create_word_features(words),'neutral'))
    
positive_tweets=[]
for string in neutral_tweets:
    words=nltk.word_tokenize(string.decode("utf-8"))
    positive_tweets.append((create_word_features(words),'positive'))
    
test_set=positive_tweets+neutral_tweets+negative_tweets


In [46]:
list_data = []
reader = csv.reader(open("datatrain.csv"),delimiter=',')
stopWords = getStopWordList('stopwordsID.txt')

for index,raw in enumerate(reader):
    if index < 1001:
        sentiment = raw[0]
        tweet = raw[1]
        tweet = processTweet(tweet)
        tweet = removeTagsAndLinks(tweet)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        mydata = (tweet,sentiment)
        list_data.append(mydata)
    else:
        pass

json_dict = {}
all_words = set(word.lower() for passage in list_data for word in word_tokenize(passage[0]))
json_dict['corpus'] = [word for word in all_words] 

with open('corpus.json', 'w') as outfile:
    json.dump(json_dict, outfile)

t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in list_data]

classifier = nltk.NaiveBayesClassifier.train(t)

f = open('sentiment.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

In [60]:
f = open('sentiment.pickle', 'rb')
classifier = pickle.load(f)
f.close()

list_tweet =  []
list_kelas = []
list_data = []

data_test = csv.reader(open('datatest.csv', 'r'), delimiter=',')
for data in data_test:
    list_data.append(data)

for data in list_data[:100]:
    tweet = data[0]
    tweet = processTweet(tweet)
    tweet = removeTagsAndLinks(tweet)
    tweet = getFeatureVector(tweet,stopWords)
    tweet = ' '.join(tweet)
    test_sent_features = {word.lower(): (word in word_tokenize(tweet.lower())) for word in all_words}
    sentiment = classifier.classify(test_sent_features)
    list_tweet.append(data[0])
    list_kelas.append(sentiment)

df = pd.DataFrame({'tweet': list_tweet,'kelas': list_kelas})
df.to_csv('hasiltest.csv', sep=';', encoding='utf-8', index=False)
test_set = csv.reader(open('hasiltest.csv', 'r'), delimiter=';')
data_list = []

for index,data in enumerate(test_set):
    if index == 0:
        pass
    else:
        sentiment = data[1]
        tweet = data[0]
        tweet = processTweet(tweet)
        tweet = removeTagsAndLinks(tweet)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        mydata = (tweet,sentiment)
        data_list.append(mydata)

d = open('corpus.json')
all_words = json.load(d)
d.close

all_words = all_words['corpus']

test_set = [({word.lower() : (word.lower() in word_tokenize(x[0])) for word in all_words}, x[1]) for x in data_list[:100]]
accuracy = nltk.classify.util.accuracy(classifier, test_set)*100
print(accuracy)

100.0


In [63]:
feature = classifier.show_most_informative_features(10)

Most Informative Features
